In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, losses

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()


In [3]:
# data normalized
x_train = x_train/255.
x_test  = x_test/255.

# reshape for model input
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [4]:
y_train = utils.to_categorical(y_train)
y_test  = utils.to_categorical(y_test)

print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [5]:
input_shape = (28,28,1) # img_rows, img_colums, color_channels
num_classes = 10

In [6]:
inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(16, kernel_size = (3, 3), activation = 'relu', padding = 'same')(inputs)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# 2nd Conv layer        
x = layers.Conv2D(16, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# Fully Connected layer        
x = layers.Flatten()(x)
x = layers.Dense(64)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 784)               0     

In [7]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Train Model
history = model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 21s 41ms/step - loss: 0.3043 - accuracy: 0.9091 - val_loss: 0.0906 - val_accuracy: 0.9726
Epoch 2/20
469/469 [==============================] - 20s 43ms/step - loss: 0.0896 - accuracy: 0.9735 - val_loss: 0.0619 - val_accuracy: 0.9795
Epoch 3/20
469/469 [==============================] - 24s 50ms/step - loss: 0.0678 - accuracy: 0.9788 - val_loss: 0.0588 - val_accuracy: 0.9812
Epoch 4/20
469/469 [==============================] - 23s 48ms/step - loss: 0.0562 - accuracy: 0.9827 - val_loss: 0.0532 - val_accuracy: 0.9804
Epoch 5/20
469/469 [==============================] - 22s 46ms/step - loss: 0.0493 - accuracy: 0.9846 - val_loss: 0.0433 - val_accuracy: 0.9850
Epoch 6/20
469/469 [==============================] - 21s 45ms/step - loss: 0.0442 - accuracy: 0.9863 - val_loss: 0.0512 - val_accuracy: 0.9828
Epoch 7/20
469/469 [==============================] - 21s 45ms/step - loss: 0.0397 - accuracy: 0.9878 - val_loss: 0.0498 - val_accuracy:

In [ ]:
# import pickle
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

In [ ]:
model.save

<bound method Model.save of <keras.engine.functional.Functional object at 0x00000202BC519B50>>

In [ ]:
score = model.evaluate(x_test, y_test)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

313/313 [==============================] - 2s 6ms/step - loss: 0.0503 - accuracy: 0.9855
Test loss:  0.050292808562517166
Test accuracy:  0.9854999780654907


In [ ]:
y_pred = model.predict(x_test[0].reshape(-1,28,28,1)).argmax(axis=1)
print(y_pred)

[7]


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("mnist_cnn_quantized.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\laksh\AppData\Local\Temp\tmp1gv6669b\assets


INFO:tensorflow:Assets written to: C:\Users\laksh\AppData\Local\Temp\tmp1gv6669b\assets


216140

In [ ]:
from tinymlgen import port
c_code = port(model, variable_name='mnist_cnn', pretty_print=True, optimize=False) 

with open('mnist_cnn.h', 'w') as f:
    print(c_code, file=f)

INFO:tensorflow:Assets written to: C:\Users\laksh\AppData\Local\Temp\tmpl3b9qk0_\assets


INFO:tensorflow:Assets written to: C:\Users\laksh\AppData\Local\Temp\tmpl3b9qk0_\assets


In [ ]:
# load raw x_test
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
    
with open('x_test.h', 'w') as f:
    print("unsigned char x_test_dat[784] = {", file=f)    
    for i in range(28):
        s = "       "
        for j in range(28):
            s+=str(x_test[0][i][j])+', ' # select x_test[0]
        print(s, file=f)
    print("};", file=f)
f.close()

In [ ]:
!type x_test.h

unsigned char x_test_dat[784] = {
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 84, 185, 159, 151, 60, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 222, 254, 254, 254, 254, 241, 198, 198, 198, 198, 198, 198, 198, 198, 170, 52, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 67, 114, 72, 114, 163, 227, 254, 225, 254, 254, 254, 250, 229, 254, 254, 

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras


In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 16)        2320      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 784)               0   

In [ ]:
model.save_weights('./weights')

In [ ]:
model.save("my_new_model")

INFO:tensorflow:Assets written to: my_new_model\assets


INFO:tensorflow:Assets written to: my_new_model\assets


In [ ]:

tf.keras.layers.Layer.get_weights(model) 



[array([[[[-2.11373419e-02,  2.08480105e-01,  3.54687184e-01,
           -4.25437421e-01, -9.15133432e-02,  9.09519661e-03,
            9.92582515e-02,  1.25714168e-01,  2.27943882e-01,
           -3.51029545e-01, -4.22855876e-02, -2.56572425e-01,
           -3.22076499e-01, -2.13466093e-01,  2.32906565e-01,
            4.93947044e-02]],
 
         [[ 1.25047594e-01,  1.06575377e-01,  6.46791384e-02,
           -4.44247067e-01, -7.22718984e-02, -5.66302240e-01,
            2.38632843e-01,  2.11558416e-01,  2.26826951e-01,
           -3.59293193e-01, -2.80819228e-03,  3.10373932e-01,
           -2.26321653e-01, -8.22089091e-02, -1.91515043e-01,
            6.10441482e-03]],
 
         [[-1.23489425e-01, -1.26826733e-01, -1.84312820e-01,
           -2.34878197e-01,  1.41237214e-01, -3.75405490e-01,
            7.15084374e-02,  1.66370720e-01,  3.56187284e-01,
            1.07922308e-01,  2.15172753e-01,  3.38611394e-01,
            1.75090075e-01,  1.21806212e-01, -1.26216710e-01,
      

In [ ]:
tflite_interpreter = tf.lite.Interpreter(model_path='mnist_cnn_quantized.tflite')
tflite_interpreter.allocate_tensors()

'''
Check input/output details
'''
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: serving_default_input_3:0
shape: [ 1 28 28  1]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.float32'>
